<a href="https://colab.research.google.com/github/valerio-unifei/ECAA07/blob/main/ECAA07_04_Manipula%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sakila Database Schema

<img src="https://github.com/valerio-unifei/ECAA07/raw/main/Bancos/sakila_schema.png">

# Baixando e conectando ao banco de dados

In [ ]:
# baixando banco
!wget https://github.com/valerio-unifei/ECAA07/raw/main/Bancos/sakila.db

In [ ]:
#conectando ao banco
import sqlite3
import pandas as pd
conn = sqlite3.connect('sakila.db')

#cursor para execução de comandos SQL
cursor = conn.cursor()

#exibindo componentes do banco
sql ='''
  SELECT type, name, sql
  FROM sqlite_master
  WHERE type="table" or type="view"
  ORDER BY type, name
  '''

pd.read_sql_query(sql=sql,con=conn)

#Visualizações

In [ ]:
pd.read_sql_query(sql='SELECT * FROM film_list',con=conn)

In [ ]:
pd.read_sql_query(sql='SELECT * FROM sales_by_store',con=conn)

In [ ]:
pd.read_sql_query(sql='SELECT * FROM sales_by_film_category ORDER BY total_sales DESC',con=conn)

# Inserindo um novo filme

Exemplo:

https://en.wikipedia.org/wiki/Once_Upon_a_Time_in_Hollywood

## Verificando se o filme existe na base

In [ ]:
#filme existe no repositório?
pd.read_sql_query(sql='SELECT * FROM film WHERE title = "ONCE UPON A TIME... IN HOLLYWOOD" or title LIKE "ONCE UPON%" ',con=conn)

## Inserindo filme

Se aparecer o erro: 

*OperationalError: database is locked*

Vá na opção: Ambiente de execução >> Redefinir ambiente de execução ...

Reconecte o ambiente do colab

In [ ]:
pd.read_sql_query(sql='SELECT * FROM film',con=conn).head(2)

In [ ]:
#inserindo filme
cursor.execute('''
  INSERT INTO film 
  
  (film_id, title, 
  description, 
  release_year, language_id, rental_duration, rental_rate, length, replacement_cost, rating, special_features, last_update)

  VALUES
  
  (1001, "Once Upon a Time In...Hollywood",
  "Actor Rick Dalton gained fame and fortune by starring in a 1950s television Western, but is now struggling to find meaningful work in a Hollywood that he doesn't recognize anymore. He spends most of his time drinking and palling around with Cliff Booth, his easygoing best friend and longtime stunt double. Rick also happens to live next door to Roman Polanski and Sharon Tate -- the filmmaker and budding actress whose futures will forever be altered by members of the Manson Family.",
  2019, 1, 3, 3.99, 162, 17.99, "R", "Trailers,Behind the Scenes", CURRENT_TIMESTAMP)

  ''')

film_id = cursor.lastrowid
print('film_id =',film_id)
pd.read_sql_query(sql='SELECT * FROM film',con=conn).tail(1)

## Inserindo categoria do filme

In [ ]:
display(pd.read_sql_query(sql='SELECT * FROM category',con=conn))
display(pd.read_sql_query(sql='SELECT * FROM film_category',con=conn).head(2))

In [ ]:
cursor.execute(f'DELETE FROM film_category WHERE film_id = {film_id}')

#categorias do filme: Drama(category_id=7), Comedy(5)
sqls = [
        f'INSERT INTO film_category (film_id, category_id, last_update) VALUES ({film_id}, 7, CURRENT_TIMESTAMP)',
        f'INSERT INTO film_category (film_id, category_id, last_update) VALUES ({film_id}, 5, CURRENT_TIMESTAMP)'
]

for sql in sqls:
  cursor.execute(sql)
  print('sql = ',sql,';')
pd.read_sql_query(sql='SELECT * FROM film_category',con=conn).tail(len(sqls))

## Inserindo atores

In [ ]:
pd.read_sql_query(sql='SELECT * FROM actor',con=conn)

In [ ]:
sqls = [
        'INSERT INTO actor VALUES (201, "Leonardo", "DiCaprio", CURRENT_TIMESTAMP)',
        'INSERT INTO actor VALUES (202, "Brad", "Pitt", CURRENT_TIMESTAMP)',
        'INSERT INTO actor VALUES (203, "Margot", "Robbie", CURRENT_TIMESTAMP)',
        'INSERT INTO actor VALUES (204, "Emile", "Hirsch", CURRENT_TIMESTAMP)',
        'INSERT INTO actor VALUES (205, "Margaret", "Qualley", CURRENT_TIMESTAMP)',
]

for sql in sqls:
  cursor.execute(sql)
  print('sql = ',sql,';')
pd.read_sql_query(sql='SELECT * FROM actor',con=conn).tail(len(sqls))

In [ ]:
pd.read_sql_query(sql='SELECT * FROM film_actor',con=conn).tail(2)

In [ ]:
cursor.execute(f'DELETE FROM film_actor WHERE film_id = {film_id}')

sqls = [
        f'INSERT INTO film_actor VALUES (201, {film_id}, CURRENT_TIMESTAMP)',
        f'INSERT INTO film_actor VALUES (202, {film_id}, CURRENT_TIMESTAMP)',
        f'INSERT INTO film_actor VALUES (203, {film_id}, CURRENT_TIMESTAMP)',
        f'INSERT INTO film_actor VALUES (204, {film_id}, CURRENT_TIMESTAMP)',
        f'INSERT INTO film_actor VALUES (205, {film_id}, CURRENT_TIMESTAMP)',
]

for sql in sqls:
  cursor.execute(sql)
  print('sql = ',sql,';')
pd.read_sql_query(sql='SELECT * FROM film_actor',con=conn).tail(len(sqls))

##Inserindo Inventário

In [ ]:
pd.read_sql_query(sql='SELECT * FROM inventory',con=conn).tail(10)

In [ ]:
cursor.execute(f'DELETE FROM inventory WHERE film_id = {film_id}')

inventory_id = 4581

for i in range(5):
  sql = f'INSERT INTO inventory VALUES ({inventory_id+1}, {film_id}, 1,CURRENT_TIMESTAMP)'
  cursor.execute(sql)
  print('sql = ',sql,';')
  inventory_id = cursor.lastrowid

for i in range(3):
  sql = f'INSERT INTO inventory VALUES ({inventory_id+1}, {film_id}, 2,CURRENT_TIMESTAMP)'
  cursor.execute(sql)
  print('sql = ',sql,';')
  inventory_id = cursor.lastrowid

pd.read_sql_query(sql='SELECT * FROM inventory',con=conn).tail(10)

# Atualizando Banco

In [ ]:
sql = '''
  SELECT *
  FROM address as a INNER JOIN city as c ON a.city_id = c.city_id
    INNER JOIN country as p ON c.country_id = p.country_id
  WHERE p.country LIKE 'B%'
  ORDER BY p.country, c.city
'''

pd.read_sql_query(sql=sql,con=conn).head()

Inserir no final de cada address o texto ' (old)' para city_id abaixo ou igual a 100.


In [ ]:
sql = 'UPDATE address SET address = address || " (old)" WHERE city_id <= 100 and address NOT LIKE "% (old)"'
cursor.execute(sql)
pd.read_sql_query(sql='SELECT * FROM address WHERE city_id <= 100',con=conn).head(100)